<a href="https://colab.research.google.com/github/ninasujit2016/natural-language-processing/blob/master/Copy_of_week5_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final project: StackOverflow assistant bot

Congratulations on coming this far and solving the programming assignments! In this final project, we will combine everything we have learned about Natural Language Processing to construct a *dialogue chat bot*, which will be able to:
* answer programming-related questions (using StackOverflow dataset);
* chit-chat and simulate dialogue on all non programming-related questions.

For a chit-chat mode we will use a pre-trained neural network engine available from [ChatterBot](https://github.com/gunthercox/ChatterBot).
Those who aim at honor certificates for our course or are just curious, will train their own models for chit-chat.
![](https://imgs.xkcd.com/comics/twitter_bot.png)
©[xkcd](https://xkcd.com)

### Data description

To detect *intent* of users questions we will need two text collections:
- `tagged_posts.tsv` — StackOverflow posts, tagged with one programming language (*positive samples*).
- `dialogues.tsv` — dialogue phrases from movie subtitles (*negative samples*).


In [3]:
! wget https://raw.githubusercontent.com/hse-aml/natural-language-processing/master/setup_google_colab.py -O setup_google_colab.py
import setup_google_colab
setup_google_colab.setup_project()

import sys
sys.path.append("..")
from common.download_utils import download_project_resources

download_project_resources()

--2020-04-20 07:26:17--  https://raw.githubusercontent.com/hse-aml/natural-language-processing/master/setup_google_colab.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1991 (1.9K) [text/plain]
Saving to: ‘setup_google_colab.py’

setup_google_colab. 100%[===================>]   1.94K  --.-KB/s    in 0s      

2020-04-20 07:26:17 (32.4 MB/s) - ‘setup_google_colab.py’ saved [1991/1991]



For those questions, that have programming-related intent, we will proceed as follow predict programming language (only one tag per question allowed here) and rank candidates within the tag using embeddings.
For the ranking part, you will need:
- `word_embeddings.tsv` — word embeddings, that you  trained with StarSpace in the 3rd assignment. It's not a problem if you didn't do it, because we can offer an alternative solution for you.

As a result of this notebook, you should obtain the following new objects that you will then use in the running bot:

- `intent_recognizer.pkl` — intent recognition model;
- `tag_classifier.pkl` — programming language classification model;
- `tfidf_vectorizer.pkl` — vectorizer used during training;
- `thread_embeddings_by_tags` — folder with thread embeddings, arranged by tags.
    

Some functions will be reused by this notebook and the scripts, so we put them into *utils.py* file. Don't forget to open it and fill in the gaps!

In [4]:
from utils import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Part I. Intent and language recognition

We want to write a bot, which will not only **answer programming-related questions**, but also will be able to **maintain a dialogue**. We would also like to detect the *intent* of the user from the question (we could have had a 'Question answering mode' check-box in the bot, but it wouldn't fun at all, would it?). So the first thing we need to do is to **distinguish programming-related questions from general ones**.

It would also be good to predict which programming language a particular question referees to. By doing so, we will speed up question search by a factor of the number of languages (10 here), and exercise our *text classification* skill a bit. :)

In [0]:
import numpy as np
import pandas as pd
import pickle
import re

from sklearn.feature_extraction.text import TfidfVectorizer

### Data preparation

In the first assignment (Predict tags on StackOverflow with linear models), you have already learnt how to preprocess texts and do TF-IDF tranformations. Reuse your code here. In addition, you will also need to [dump](https://docs.python.org/3/library/pickle.html#pickle.dump) the TF-IDF vectorizer with pickle to use it later in the running bot.

In [0]:
def tfidf_features(X_train, X_test, vectorizer_path):
    """Performs TF-IDF transformation and dumps the model."""
    
    # Train a vectorizer on X_train data.
    # Transform X_train and X_test data.
    
    # Pickle the trained vectorizer to 'vectorizer_path'
    # Don't forget to open the file in writing bytes mode.
    
    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    tfidf_vectorizer = TfidfVectorizer(token_pattern='(\S+)', min_df=5, max_df=0.9, ngram_range=(1,2),
                                       binary=False, norm='l2', use_idf=True)
    tfidf_vectorizer.fit(X_train)
    X_train = tfidf_vectorizer.transform(X_train)
    X_test = tfidf_vectorizer.transform(X_test)
    
    with open(vectorizer_path, 'wb') as f:
        pickle.dump(tfidf_vectorizer, f)
    return X_train, X_test

Now, load examples of two classes. Use a subsample of stackoverflow data to balance the classes. You will need the full data later.

In [0]:
sample_size = 200000

dialogue_df = pd.read_csv('data/dialogues.tsv', sep='\t').sample(sample_size, random_state=0)
stackoverflow_df = pd.read_csv('data/tagged_posts.tsv', sep='\t').sample(sample_size, random_state=0)

In [150]:
len(dialogue_df)

200000

Check how the data look like:

In [151]:
dialogue_df.head()

,text,tag
82925,"Donna, you are a muffin.",dialogue
48774,He was here last night till about two o'clock....,dialogue
55394,"All right, then make an appointment with her s...",dialogue
90806,"Hey, what is this-an interview? We're supposed...",dialogue
107758,Yeah. He's just a friend of mine I was trying ...,dialogue


In [152]:
stackoverflow_df.head()

,post_id,title,tag
2168983,43837842,Efficient Algorithm to compose valid expressio...,python
1084095,15747223,Why does this basic thread program fail with C...,c_cpp
1049020,15189594,Link to scroll to top not working,javascript
200466,3273927,Is it possible to implement ping on windows ph...,c#
1200249,17684551,GLSL normal mapping issue,c_cpp


Apply *text_prepare* function to preprocess the data:

In [0]:
from utils import text_prepare

In [0]:
dialogue_df['text'] = dialogue_df['text'].apply(text_prepare)  #[text_prepare(x) for x in dialogue_df['text']]  ######### YOUR CODE HERE #############
stackoverflow_df['title'] =stackoverflow_df['title'].apply(text_prepare) #[text_prepare(tit) for tit in stackoverflow_df['title']]  ######### YOUR CODE HERE #############

In [155]:
list(stackoverflow_df['title'])[1:6]

['basic thread program fail clang pass g++',
 'link scroll top working',
 'possible implement ping windows phone 7',
 'glsl normal mapping issue',
 'correct syntax main method']

### Intent recognition

We will do a binary classification on TF-IDF representations of texts. Labels will be either `dialogue` for general questions or `stackoverflow` for programming-related questions. First, prepare the data for this task:
- concatenate `dialogue` and `stackoverflow` examples into one sample
- split it into train and test in proportion 9:1, use *random_state=0* for reproducibility
- transform it into TF-IDF features

In [0]:
from sklearn.model_selection import train_test_split

In [157]:
dialogue_df['text'].values.shape[0]


200000

In [158]:
X = np.concatenate([dialogue_df['text'].values, stackoverflow_df['title'].values])
y = ['dialogue'] * dialogue_df.shape[0] + ['stackoverflow'] * stackoverflow_df.shape[0]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1,random_state=0)######### YOUR CODE HERE ##########
print('Train size = {}, test size = {}'.format(len(X_train), len(X_test)))

X_train_tfidf, X_test_tfidf =tfidf_features(X_train,X_test, './tfidf_vectorizer.pkl' )######### YOUR CODE HERE ###########

Train size = 360000, test size = 40000


Train the **intent recognizer** using LogisticRegression on the train set with the following parameters: *penalty='l2'*, *C=10*, *random_state=0*. Print out the accuracy on the test set to check whether everything looks good.

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [160]:
######################################
######### YOUR CODE HERE #############
######################################
intent_recognizer= LogisticRegression(penalty='l2', C=10, random_state=0,max_iter=500)
intent_recognizer.fit(X_train_tfidf,y_train)
#predictions=model.predict(X_test_tfidf)
#accuracy_score(y_test,predictions)

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [161]:
# Check test accuracy.
y_test_pred = intent_recognizer.predict(X_test_tfidf)
test_accuracy = accuracy_score(y_test, y_test_pred)
print('Test accuracy = {}'.format(test_accuracy))

Test accuracy = 0.991575


Dump the classifier to use it in the running bot.

In [0]:
pickle.dump(intent_recognizer, open(RESOURCE_PATH['INTENT_RECOGNIZER'], 'wb'))

In [166]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.22.2.post1.


In [0]:
#!ls ./
from google.colab import files
files.download('intent_recognizer.pkl')

In [163]:
!ls -lh ./*.pkl

-rw-r--r-- 1 root root 529K Apr 20 10:53 ./intent_recognizer.pkl
-rw-r--r-- 1 root root 5.2M Apr 20 08:07 ./tag_classifier.pkl
-rw-r--r-- 1 root root  29M Apr 20 10:51 ./tfidf_vectorizer.pkl


In [0]:
import pickle
with open('intent_recognizer.pkl','rb') as f:
      xxfpk=pickle.load(f)


### Programming language classification 

We will train one more classifier for the programming-related questions. It will predict exactly one tag (=programming language) and will be also based on Logistic Regression with TF-IDF features. 

First, let us prepare the data for this task.

In [0]:
X = stackoverflow_df['title'].values
y = stackoverflow_df['tag'].values

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print('Train size = {}, test size = {}'.format(len(X_train), len(X_test)))

Train size = 160000, test size = 40000


Let us reuse the TF-IDF vectorizer that we have already created above. It should not make a huge difference which data was used to train it.

In [0]:
vectorizer = pickle.load(open(RESOURCE_PATH['TFIDF_VECTORIZER'], 'rb'))

X_train_tfidf, X_test_tfidf = vectorizer.transform(X_train), vectorizer.transform(X_test)

Train the **tag classifier** using OneVsRestClassifier wrapper over LogisticRegression. Use the following parameters: *penalty='l2'*, *C=5*, *random_state=0*.

In [0]:
from sklearn.multiclass import OneVsRestClassifier

In [36]:
######################################
######### YOUR CODE HERE #############
######################################
tag_classifier=OneVsRestClassifier(LogisticRegression(penalty='l2',random_state=0,C=5,max_iter=500))
tag_classifier.fit(X_train_tfidf,y_train)

OneVsRestClassifier(estimator=LogisticRegression(C=5, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=500,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=0, solver='lbfgs',
                                                 tol=0.0001, verbose=0,
                                                 warm_start=False),
                    n_jobs=None)

In [37]:
# Check test accuracy.
y_test_pred = tag_classifier.predict(X_test_tfidf)
test_accuracy = accuracy_score(y_test, y_test_pred)
print('Test accuracy = {}'.format(test_accuracy))

Test accuracy = 0.8007


Dump the classifier to use it in the running bot.

In [0]:
pickle.dump(tag_classifier, open(RESOURCE_PATH['TAG_CLASSIFIER'], 'wb'))

In [0]:
files.download('tag_classifier.pkl')

## Part II. Ranking  questions with embeddings

To find a relevant answer (a thread from StackOverflow) on a question you will use vector representations to calculate similarity between the question and existing threads. We already had `question_to_vec` function from the assignment 3, which can create such a representation based on word vectors. 

However, it would be costly to compute such a representation for all possible answers in *online mode* of the bot (e.g. when bot is running and answering questions from many users). This is the reason why you will create a *database* with pre-computed representations. These representations will be arranged by non-overlaping tags (programming languages), so that the search of the answer can be performed only within one tag each time. This will make our bot even more efficient and allow not to store all the database in RAM. 

Load StarSpace embeddings which were trained on Stack Overflow posts. These embeddings were trained in *supervised mode* for duplicates detection on the same corpus that is used in search. We can account on that these representations will allow us to find closely related answers for a question. 

If for some reasons you didn't train StarSpace embeddings in the assignment 3, you can use [pre-trained word vectors](https://code.google.com/archive/p/word2vec/) from Google. All instructions about how to work with these vectors were provided in the same assignment. However, we highly recommend to use StarSpace's embeddings, because it contains more appropriate embeddings. If you chose to use Google's embeddings, delete the words, which is not in Stackoverflow data.

In [52]:
files.upload()

Saving utils.py to utils.py


{'utils.py': b'import nltk\nimport pickle\nimport re\nimport numpy as np\n\nnltk.download(\'stopwords\')\nfrom nltk.corpus import stopwords\n\n# Paths for all resources for the bot.\nRESOURCE_PATH = {\n    \'INTENT_RECOGNIZER\': \'intent_recognizer.pkl\',\n    \'TAG_CLASSIFIER\': \'tag_classifier.pkl\',\n    \'TFIDF_VECTORIZER\': \'tfidf_vectorizer.pkl\',\n    \'THREAD_EMBEDDINGS_FOLDER\': \'thread_embeddings_by_tags\',\n    \'WORD_EMBEDDINGS\': \'word_embeddings.tsv\',\n}\n\n\ndef text_prepare(text):\n    """Performs tokenization and simple preprocessing."""\n\n    replace_by_space_re = re.compile(\'[/(){}\\[\\]\\|@,;]\')\n    bad_symbols_re = re.compile(\'[^0-9a-z #+_]\')\n    stopwords_set = set(stopwords.words(\'english\'))\n\n    text = text.lower()\n    text = replace_by_space_re.sub(\' \', text)\n    text = bad_symbols_re.sub(\'\', text)\n    text = \' \'.join([x for x in text.split() if x and x not in stopwords_set])\n\n    return text.strip()\n\n\ndef load_embeddings(embedding

In [0]:
def question_to_vec(question, embeddings, dim):
    """Transforms a string to an embedding by averaging word embeddings."""

    # Hint: you have already implemented exactly this function in the 3rd assignment.
    #### YOUR CODE HERE ####
    question_list=question.strip().split()
    if question=='' or all(word not in embeddings for word in question_list) :
        return np.zeros(dim)
    else:
    #newlist=[word if word in embeddings for word in question_list]\n",
        embs=[embeddings[word] for word in question_list if word in embeddings]
        if embs!=[]:
            return sum(embs)/len(embs)
        else:
            return np.zeros(dim) 


In [44]:
#setup_google_colab.setup_week3()
from common.download_utils import download_week3_resources
download_week3_resources()

In [45]:
def prepare_file(in_, out_):
    out = open(out_, 'w')
    for line in open(in_, encoding='utf8'):
        line = line.strip().split('\t')
        new_line = [text_prepare(q) for q in line]
        print(*new_line, sep='\t', file=out)
    out.close()
prepare_file('data/train.tsv','data/train_prepared.tsv')
!ls ./data/

dialogues.tsv	  test_embeddings.tsv  train_prepared.tsv  validation.tsv
tagged_posts.tsv  test.tsv	       train.tsv


In [46]:
!./Starspace/starspace train -trainFile ./data/train_prepared.tsv -model SS_starspace -trainMode 3 -adagrad true -ngrams 1 -epoch 5 -dim 100 -similarity "cosine" -minCount 2 -verbose true  -fileFormat labelDoc -negSearchLimit 10 -thread 10 -lr 0.05
#starspace_embeddings, embeddings_dim = load_embeddings('data/word_embeddings.tsv')

Arguments: 
lr: 0.05
dim: 100
epoch: 5
maxTrainTime: 8640000
validationPatience: 10
saveEveryEpoch: 0
loss: hinge
margin: 0.05
similarity: cosine
maxNegSamples: 10
negSearchLimit: 10
batchSize: 5
thread: 10
minCount: 2
minCountLabel: 1
label: __label__
label: __label__
ngrams: 1
bucket: 2000000
adagrad: 1
trainMode: 3
fileFormat: labelDoc
normalizeText: 0
dropoutLHS: 0
dropoutRHS: 0
useWeight: 0
weightSep: :
Start to initialize starspace model.
Build dict from input file : ./data/train_prepared.tsv
Read 12M words
Number of words in dictionary:  95058
Number of labels in dictionary: 0
Loading data from file : ./data/train_prepared.tsv
Total number of examples loaded : 999740
Initialized model weights. Model size :
matrix : 95058 100
Training epoch 0: 0.05 0.01
Epoch: 100.0%  lr: 0.040000  loss: 0.043449  eta: 0h13m  tot: 0h3m15s  (20.0%)
 ---+++                Epoch    0 Train error : 0.04357778 +++--- ☃
Training epoch 1: 0.04 0.01
Epoch: 100.0%  lr: 0.030000  loss: 0.013287  eta: 0h9m 

In [66]:
!rm utils.py 'utils (1).py'
files.upload()

Saving utils.py to utils.py


{'utils.py': b'import nltk\nimport pickle\nimport re\nimport numpy as np\n\nnltk.download(\'stopwords\')\nfrom nltk.corpus import stopwords\n\n# Paths for all resources for the bot.\nRESOURCE_PATH = {\n    \'INTENT_RECOGNIZER\': \'intent_recognizer.pkl\',\n    \'TAG_CLASSIFIER\': \'tag_classifier.pkl\',\n    \'TFIDF_VECTORIZER\': \'tfidf_vectorizer.pkl\',\n    \'THREAD_EMBEDDINGS_FOLDER\': \'thread_embeddings_by_tags\',\n    \'WORD_EMBEDDINGS\': \'word_embeddings.tsv\',\n}\n\n\ndef text_prepare(text):\n    """Performs tokenization and simple preprocessing."""\n\n    replace_by_space_re = re.compile(\'[/(){}\\[\\]\\|@,;]\')\n    bad_symbols_re = re.compile(\'[^0-9a-z #+_]\')\n    stopwords_set = set(stopwords.words(\'english\'))\n\n    text = text.lower()\n    text = replace_by_space_re.sub(\' \', text)\n    text = bad_symbols_re.sub(\'\', text)\n    text = \' \'.join([x for x in text.split() if x and x not in stopwords_set])\n\n    return text.strip()\n\n\ndef load_embeddings(embedding

In [0]:
from utils import *
def load_embeddings(embeddings_path):
    """Loads pre-trained word embeddings from tsv file.

    Args:
      embeddings_path - path to the embeddings file.

    Returns:
      embeddings - dict mapping words to vectors;
      embeddings_dim - dimension of the vectors.
    """
    
    # Hint: you have already implemented a similar routine in the 3rd assignment.
    # Note that here you also need to know the dimension of the loaded embeddings.
    # When you load the embeddings, use numpy.float32 type as dtype
    embeddings={}
    with open(embeddings_path,'r') as f:
        for row in f:
            splitline = row.strip().split()
            word=row.strip().split()[0]
            vector=np.array([float(val) for val in splitline[1:]],dtype=np.float32)
            embeddings[word]=vector
        embeddings_dim=vector.shape[0]
    return embeddings, embeddings_dim
starspace_embeddings, embeddings_dim = load_embeddings('SS_starspace.tsv')

In [74]:
embeddings_dim

100

Since we want to precompute representations for all possible answers, we need to load the whole posts dataset, unlike we did for the intent classifier:

In [0]:
posts_df = pd.read_csv('data/tagged_posts.tsv', sep='\t')

Look at the distribution of posts for programming languages (tags) and find the most common ones. 
You might want to use pandas [groupby](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html) and [count](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.count.html) methods:

In [76]:
counts_by_tag = posts_df.groupby('tag').count()  ######### YOUR CODE HERE #############
#counts_by_tag = posts_df['tag'].value_counts().to_dict()
counts_by_tag

,post_id,title
tag,,
c#,394451,394451
c_cpp,281300,281300
java,383456,383456
javascript,375867,375867
php,321752,321752
python,208607,208607
r,36359,36359
ruby,99930,99930
swift,34809,34809


Now for each `tag` you need to create two data structures, which will serve as online search index:
* `tag_post_ids` — a list of post_ids with shape `(counts_by_tag[tag],)`. It will be needed to show the title and link to the thread;
* `tag_vectors` — a matrix with shape `(counts_by_tag[tag], embeddings_dim)` where embeddings for each answer are stored.

Implement the code which will calculate the mentioned structures and dump it to files. It should take several minutes to compute it.

In [0]:
#files.download('SS_starspace.tsv')
!mv SS_starspace.tsv word_embeddings.tsv

In [109]:
counts_by_tag['title'].tolist()[:10]
#counts_by_tag.head()

[394451, 281300, 383456, 375867, 321752, 208607, 36359, 99930, 34809, 35044]

In [0]:
import os
os.makedirs(RESOURCE_PATH['THREAD_EMBEDDINGS_FOLDER'], exist_ok=True)
ii=0
for tag, count in counts_by_tag.items():
    #print(tag,count)
    tag_posts = posts_df[posts_df['tag'] == tag]

    tag_post_ids =tag_posts['post_id'].values  ######### YOUR CODE HERE #############
    count1=count.tolist()[0]
    tag_vectors = np.zeros((count1, embeddings_dim), dtype=np.float32)
    for i, title in enumerate(tag_posts['title']):
        tag_vectors[i,:] = question_to_vec(title, starspace_embeddings,embeddings_dim)######### YOUR CODE HERE #############

    # Dump post ids and vectors to a file.
    filename = os.path.join(RESOURCE_PATH['THREAD_EMBEDDINGS_FOLDER'], os.path.normpath('%s.pkl' % tag))
    pickle.dump((tag_post_ids, tag_vectors), open(filename, 'wb'))  

In [0]:
#!ls -lh ./thread_embeddings_by_tags
#!gzip ./thread_embeddings_by_tags/*.pkl

#files.download('./thread_embeddings_by_tags/post_id.pkl.gz')
files.download('./thread_embeddings_by_tags/title.pkl.gz')

In [138]:
!ls ./

boost_1_63_0.zip		       setup_google_colab.py
common				       SS_starspace
data				       Starspace
dialogue_manager.py		       tag_classifier.pkl
GoogleNews-vectors-negative300.bin.gz  tfidf_vectorizer.pkl
main_bot.py			       thread_embeddings_by_tags
__pycache__			       utils.py
sample_data			       word_embeddings.tsv


In [125]:
n=0
#tag_vectors = np.zeros((count, embeddings_dim), dtype=np.float32)
for tag, count in counts_by_tag.iterrows():
    if n<=2:
        print(count.tolist()[0])
     # tag_vectors[i,:] = question_to_vec(title, starspace_embeddings,embeddings_dim)
    n=n+1

394451
281300
383456
